## Trapezoidal

$$ \int_{a}^{b} f(x) dx = \frac{b-a}{2n} [f(a)+2 \sum_{i=1}^{n-1} f(a+ih) +f(b)] $$

In [ ]:
def trapezoid(a, b, n):
    h = (b - a) / n
    s = (y(a) + y(b))
    i = 1
    for i in range(1, n):
        s += 2 * y(a + i * h)
        i += 1
    return ((h / 2) * s)



## Midpoint rule

$$ \int_{a}^{b} f(x) dx = h \sum_{i=0}^{n-1} f(a+(i+\frac{1}{2})h) +f(b) $$

In [ ]:
def midpoint(a, b, n):
    h = (b - a) / n
    total = 0
    for i in range(n):
        x_mid = a + (i + 0.5) * h
        total += y(x_mid)
    return h * total


## Simpson 1/3 

$$ \int_{a}^{b} f(x) dx = \frac{h}{3} [f(a)+ 4 \sum_{i=1,3,5,...}^{n-1} f(a+ih) + 2 \sum_{i=2,4,6,...}^{n-1} f(a+ih) +f(b)] $$

In [ ]:
def simpson(a, b, n): 
    h = (b - a) / n
    s = y(a) + y(b)
    
    for i in range(1, n):
        x = a + i * h
        if i % 2 == 0:
            s += 2 * y(x)  # untuk indeks genap
        else:
            s += 4 * y(x)  # untuk indeks ganjil
    
    return (h / 3) * s

## Simpson 3/8

$$ \int_{a}^{b} f(x) dx = \frac{3h}{8} [f(a)+ 3 \sum_{i=1,2,4,5,7,8,..}^{n-1} f(a+ih)  + 2 \sum_{i=3,6,9,...}^{n-1} f(a+ih) +f(b)] $$

In [ ]:
def simpson38(a, b, n):
    h = (b - a) / n
    s = y(a) + y(b)
    
    for i in range(1, n):
        x = a + i * h
        # Jika i kelipatan 3 → dikalikan 2
        if i % 3 == 0:
            s += 2 * y(x)
        else:
            s += 3 * y(x)
    
    return (3 * h / 8) * s


## Romberg

In [ ]:
from mpi4py import MPI
import timeit
import math
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
start = MPI.Wtime()

def y(x):
    return (4/(1+x**2))

def trapezoid(a, b, n):
    h = (b - a) / n
    s = (y(a) + y(b))
    i = 1
    for i in range(1, n):
        s += 2 * y(a + i * h)
        i += 1
    return ((h / 2) * s)


def romberg(a, b, max_k=5):
    R = np.zeros((max_k, max_k))
    for k in range(max_k):
        n = 2**k
        R[k, 0] = trapezoid(y, a, b, n)
        for j in range(1, k + 1):
            R[k, j] = R[k, j-1] + (R[k, j-1] - R[k-1, j-1]) / (4**j - 1)
    return R[max_k-1, max_k-1]

a = 0
b = 1
n = 10**9
h = (b - a)/n

local_n = n // size
local_a = a + rank * local_n * h
local_b = local_a + local_n * h

local_integral = romberg(local_a, local_b, local_n)

my_integral=0
if rank == 0:
    my_integral = local_integral
    for i in range(1,size):
        my_integral_2 = comm.recv(source=MPI.ANY_SOURCE)
        my_integral = my_integral + my_integral_2
    print("hasil numerik = ", my_integral)
    print("waktu = ", MPI.Wtime()-start, "detik")
else:
    comm.send(local_integral,dest=0)

## Simpson 1/3 dan 3/8

In [ ]:
def simpson_mix(a, b, n):
    if n < 2:
        raise ValueError("n minimal 2 agar metode Simpson dapat diterapkan.")

    h = (b - a) / n
    s = 0

    def f(x):
        return y(x)

    # Jika n kelipatan 2 (tapi bukan kelipatan 3)
    if n % 2 == 0:
        s = f(a) + f(b)
        for i in range(1, n):
            x = a + i * h
            s += 4 * f(x) if i % 2 == 1 else 2 * f(x)
        return (h / 3) * s

    # Jika n kelipatan 3
    elif n % 3 == 0:
        s = f(a) + f(b)
        for i in range(1, n):
            x = a + i * h
            s += 3 * f(x) if i % 3 != 0 else 2 * f(x)
        return (3 * h / 8) * s

    # Jika n tidak kelipatan 2 atau 3 → gabungan
    else:
        n1 = n - 3  # bagian awal (Simpson 1/3)
        b1 = a + n1 * h
        s1 = f(a) + f(b1)
        for i in range(1, n1):
            x = a + i * h
            s1 += 4 * f(x) if i % 2 == 1 else 2 * f(x)
        result_13 = (h / 3) * s1

        # Bagian terakhir (3 segmen pakai 3/8 rule)
        s2 = f(b1) + f(b)
        for i in range(1, 3):
            s2 += 3 * f(b1 + i * h)
        result_38 = (3 * h / 8) * s2

        return result_13 + result_38


## Jacobi 

In [ ]:
from mpi4py import MPI
import numpy as np

def jacobi_parallel(A, b, tol=1e-6, max_iter=100):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    n = len(b)
    rows_per_proc = n // size
    start = rank * rows_per_proc
    end = n if rank == size - 1 else (rank + 1) * rows_per_proc

    # Bagi matriks dan vektor
    A_local = A[start:end, :]
    b_local = b[start:end]

    # Inisialisasi solusi
    x = np.zeros(n)
    x_new_local = np.zeros(len(b_local))

    for k in range(max_iter):
        # Broadcast solusi lama ke semua proses
        x_old = comm.bcast(x, root=0)

        # Hitung x lokal (bagian masing-masing proses)
        for i_local, i in enumerate(range(start, end)):
            s = np.dot(A_local[i_local, :], x_old) - A_local[i_local, i] * x_old[i]
            x_new_local[i_local] = (b_local[i_local] - s) / A_local[i_local, i]

        # Gabungkan hasil semua proses
        x_new = np.zeros_like(x)
        comm.Allgather(x_new_local, x_new)

        # Cek konvergensi
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            if rank == 0:
                print(f"Konvergen pada iterasi ke-{k+1}")
            break

        x = x_new.copy()

    return x

# -------------------------
# Eksekusi dengan mpiexec
# -------------------------
if __name__ == "__main__":
    A = np.array([[10, -1, 2, 0],
                  [-1, 11, -1, 3],
                  [2, -1, 10, -1],
                  [0, 3, -1, 8]], dtype=float)
    b = np.array([6, 25, -11, 15], dtype=float)

    x = jacobi_parallel(A, b)
    comm = MPI.COMM_WORLD
    if comm.Get_rank() == 0:
        print("Solusi akhir (Jacobi Paralel):", x)


## Monte Carlo

In [ ]:
def monte_carlo(f, a, b, n):
    x = np.random.uniform(a, b, n)
    fx = f(x)
    return (b - a) * np.mean(fx)